In [1]:
#!/usr/local/bin/python3
# assign.py : Assign people to teams
#
# Code by: name IU ID
#
# Based on skeleton code by D. Crandall and B551 Staff, September 2021
#

In [2]:
import sys
import time
import numpy as np

# Cost function

In [3]:
def grading(groups):
    n_groups = len(groups)
    
    return n_groups*5

In [4]:
def wrong_group_size(group_assigned, want_group_size, groups_size):

    assigned_group_size=[groups_size[group] for group in group_assigned]
    group_size_compare = [assigned_group_size[n]==want_group_size[n] for n in range(len(group_assigned))]
    n_wrong_groups = len(group_assigned) - np.count_nonzero(group_size_compare)
    
    return n_wrong_groups*2


In [5]:
def wanted_not_in_assigned(people_assigned, want):

    people_assigned_mask = people_assigned.copy()
    for n, row in enumerate(want):
        for m, item in enumerate(row):
            if item in ['xxx','zzz']:
                people_assigned_mask[n] = people_assigned_mask[n]+['xxx','zzz']

    match_wanted_assigned = [[want[i][n] in people_assigned_mask[i] for n in range(len(want[i]))] for i in range(len(people_assigned))]
    match_wanted_assigned = [elem for row in match_wanted_assigned for elem in row]
    
    wanted_not_assigned = len(match_wanted_assigned)- np.count_nonzero(match_wanted_assigned)
    
    return wanted_not_assigned *(0.05*60)

In [6]:
def not_wanted_in_assigned(people_assigned, not_want):    
    match_not_want_assigned = [[not_want[i][n] in people_assigned[i] for n in range(len(not_want[i]))] for i in range(len(people_assigned))]
    match_not_want_assigned = [elem for row in match_not_want_assigned for elem in row]
    not_want_assigned = np.count_nonzero(match_not_want_assigned)

    return not_want_assigned*10

# Check if the cost functions work on the example given in the instructions

In [ ]:
filename = 'test1.txt'

with open(filename) as f:
    lines = f.readlines()

survey_text = [line.rstrip('\n') for line in lines]
survey_data = np.array([entry.split() for entry in survey_text])
survey_data

array([['djcran', 'djcran-vkvats-nthakurd', 'sahmaini'],
       ['sahmaini', 'sahmaini', '_'],
       ['sulagaop', 'sulagaop-xxx-xxx', '_'],
       ['fanjun', 'fanjun-xxx', 'nthakurd'],
       ['nthakurd', 'nthakurd', 'djcran,fanjun'],
       ['vkvats', 'vkvats-sahmaini', '_']], dtype='<U22')

In [ ]:
#survey inputs
people = survey_data[:,0]
want = [item.split('-') for item in survey_data[:,1]]
not_want = [item.split(',') for item in survey_data[:,2]]
want_group_size = [len(item) for item in want]

print(f'people {people}')
print (f'want {want}')
print (f'not_want {not_want}')
print (f'want_group_size {want_group_size}')

people ['djcran' 'sahmaini' 'sulagaop' 'fanjun' 'nthakurd' 'vkvats']
want [['djcran', 'vkvats', 'nthakurd'], ['sahmaini'], ['sulagaop', 'xxx', 'xxx'], ['fanjun', 'xxx'], ['nthakurd'], ['vkvats', 'sahmaini']]
not_want [['sahmaini'], ['_'], ['_'], ['nthakurd'], ['djcran', 'fanjun'], ['_']]
want_group_size [3, 1, 3, 2, 1, 2]


In [ ]:
# assign a group
#group example from assignment instructions
groups = [['djcran','vkvats','nthakurd'], ["sahmaini"], ["sulagaop", 'fanjun']]
print(f'groups from example {groups}')


groups from example [['djcran', 'vkvats', 'nthakurd'], ['sahmaini'], ['sulagaop', 'fanjun']]


In [ ]:
# group stats
n_groups = len(groups)
groups_size = [len(group) for group in groups]

people_group_mat = [[people[x] in group for group in groups] for x in range(len(people))]
group_assigned = [people_group_mat[n].index(1) for n in range(len(people_group_mat))]

people_assigned =[groups[n] for n in group_assigned]

print (f'n_groups {n_groups}')
print(f'groups_size {groups_size}')
print (f'group_assigned {group_assigned}')
print (f'people_assigned {people_assigned}')

n_groups 3
groups_size [3, 1, 2]
group_assigned [0, 1, 2, 2, 0, 0]
people_assigned [['djcran', 'vkvats', 'nthakurd'], ['sahmaini'], ['sulagaop', 'fanjun'], ['sulagaop', 'fanjun'], ['djcran', 'vkvats', 'nthakurd'], ['djcran', 'vkvats', 'nthakurd']]


In [ ]:
#calculate cost
cost_grading = grading(groups)
print(f'grading {cost_grading}')

cost_wrong_group_size = wrong_group_size(group_assigned, want_group_size, groups_size)
print(f'cost_wrong_group_size {cost_wrong_group_size}')

cost_wanted_not_in_group = wanted_not_in_assigned(people_assigned, want)
print(f'cost_wanted_not_in_group {cost_wanted_not_in_group}')

cost_not_wanted_in_group = not_wanted_in_assigned(people_assigned, not_want)
print(f'cost_not_wanted_in_group {cost_not_wanted_in_group}')

cost = cost_grading + cost_wrong_group_size + cost_wanted_not_in_group + cost_not_wanted_in_group
print (f'cost {cost}')

grading 15
cost_wrong_group_size 6
cost_wanted_not_in_group 3.0
cost_not_wanted_in_group 10
cost 34.0


# Random search - fixed number of iterations


In [17]:
def assign_random_groups(people):

    if len(people)% 3 ==0:
        poss_n_groups = int(len(people)/3)
    else:
        poss_n_groups = int(np.floor(len(people)/3))+1

    rand_groups = np.array([np.random.randint(0,poss_n_groups+1) for x in range(len(people))])

    n_in_group =[np.count_nonzero(rand_groups == n) for n in range(poss_n_groups+1)]

    while max(n_in_group)>3:
        rand_groups = np.array([np.random.randint(0,poss_n_groups) for x in range(len(people))])
        n_in_group =[np.count_nonzero(rand_groups == n) for n in range(poss_n_groups+1)]

    assigned_groups = []
    [assigned_groups.append(list(people[np.where(rand_groups ==n)])) for n in range(poss_n_groups+1)]

    groups = assigned_groups

    try:
        groups.pop([groups[n]==[]for n in range(len(groups))].index(1))
    except:
        pass

#     print(f'groups {groups}')
    
    return groups

In [19]:
def solver(filename):
    
#     filename ='test1.txt'    

    with open(filename) as f:
        lines = f.readlines()

    survey_text = [line.rstrip('\n') for line in lines]
    survey_data = np.array([entry.split() for entry in survey_text])

    #survey inputs
    people = survey_data[:,0]
    want = [item.split('-') for item in survey_data[:,1]]
    not_want = [item.split(',') for item in survey_data[:,2]]
    want_group_size = [len(item) for item in want]



    min_cost = 100000000
    n = 1
    runs = 100
    for n in range(runs):
        #randomly generated group
        rand_groups = assign_random_groups(people)
        groups = rand_groups
#         group example from assignment instructions
#         groups = [['djcran','vkvats','nthakurd'], ["sahmaini"], ["sulagaop", 'fanjun']]
#         print(f'groups from example {groups}')

        # group stats
        n_groups = len(groups)
        groups_size = [len(group) for group in groups]

        people_group_mat = [[people[x] in group for group in groups] for x in range(len(people))]
        group_assigned = [people_group_mat[n].index(1) for n in range(len(people_group_mat))]

        people_assigned =[groups[n] for n in group_assigned]


        #calculate cost
        cost_grading = grading(groups)
        cost_wrong_group_size = wrong_group_size(group_assigned, want_group_size, groups_size)
        cost_wanted_not_in_group = wanted_not_in_assigned(people_assigned, want)
        cost_not_wanted_in_group = not_wanted_in_assigned(people_assigned, not_want)

        cost = cost_grading + cost_wrong_group_size + cost_wanted_not_in_group + cost_not_wanted_in_group


        if cost < min_cost:
            min_cost_groups = groups
            min_cost = cost
            print(n, min_cost_groups, min_cost)
        
        n+=1
        
    min_cost_groups_str = ['-'.join(x) for x in min_cost_groups]
    print (f'\nmin_cost_group after {runs} random samples is {min_cost_groups_str, min_cost}')

    result = {"assigned-groups": min_cost_groups_str,
              "total-cost" : min_cost}

    

    
    
    return result


In [42]:
solver('/content/drive/MyDrive/Colab Notebooks/test3.txt')

0 [['djcran', 'zhao', 'wang'], ['zhou', 'wang5', 'zhang'], ['sun', 'zheng', 'zhang3'], ['nthakurd', 'qian'], ['sulagaop', 'li', 'chen'], ['wu', 'feng', 'li4'], ['sahmaini', 'fanjun', 'vkvats']] 98.0
1 [['feng', 'wang5', 'zhang'], ['sahmaini', 'nthakurd', 'zhao'], ['fanjun', 'sun', 'zhou'], ['sulagaop', 'li4'], ['li', 'zheng', 'chen'], ['djcran', 'wu', 'wang'], ['vkvats', 'qian', 'zhang3']] 90.0
8 [['sulagaop', 'zheng', 'li4'], ['djcran', 'zhao', 'wu'], ['li', 'zhou'], ['vkvats', 'wang', 'wang5'], ['sahmaini', 'fanjun', 'feng'], ['nthakurd', 'qian', 'chen'], ['sun', 'zhang3', 'zhang']] 89.0
13 [['nthakurd', 'qian', 'zheng'], ['sulagaop', 'wang', 'li4'], ['zhao', 'wu', 'chen'], ['vkvats', 'feng'], ['fanjun', 'zhou', 'zhang'], ['sahmaini', 'sun', 'zhang3'], ['djcran', 'li', 'wang5']] 85.0
29 [['wu', 'wang', 'wang5'], ['vkvats', 'feng'], ['sahmaini', 'qian', 'li'], ['nthakurd', 'zhou', 'zheng'], ['sulagaop', 'fanjun', 'sun'], ['zhao', 'li4', 'zhang'], ['djcran', 'chen', 'zhang3']] 82.0
73 

{'assigned-groups': ['qian-li-zheng',
  'djcran-vkvats',
  'sulagaop-fanjun-wang5',
  'sahmaini-feng-zhang',
  'zhao-wu-li4',
  'zhou-wang-chen',
  'nthakurd-sun-zhang3'],
 'total-cost': 77.0}

# Random Search - Yield when a lower cost is found

In [ ]:
def solver_yield(filename):
    
#     filename ='test1.txt'    

    with open(filename) as f:
        lines = f.readlines()

    survey_text = [line.rstrip('\n') for line in lines]
    survey_data = np.array([entry.split() for entry in survey_text])

    #survey inputs
    people = survey_data[:,0]
    want = [item.split('-') for item in survey_data[:,1]]
    not_want = [item.split(',') for item in survey_data[:,2]]
    want_group_size = [len(item) for item in want]



    min_cost = 100000000
    while min_cost>0:
        #randomly generated group
        rand_groups = assign_random_groups(people)
        groups = rand_groups
#         group example from assignment instructions
#         groups = [['djcran','vkvats','nthakurd'], ["sahmaini"], ["sulagaop", 'fanjun']]
#         print(f'groups from example {groups}')

        # group stats
        n_groups = len(groups)
        groups_size = [len(group) for group in groups]

        people_group_mat = [[people[x] in group for group in groups] for x in range(len(people))]
        group_assigned = [people_group_mat[n].index(1) for n in range(len(people_group_mat))]

        people_assigned =[groups[n] for n in group_assigned]


        #calculate cost
        cost_grading = grading(groups)
        cost_wrong_group_size = wrong_group_size(group_assigned, want_group_size, groups_size)
        cost_wanted_not_in_group = wanted_not_in_assigned(people_assigned, want)
        cost_not_wanted_in_group = not_wanted_in_assigned(people_assigned, not_want)

        cost = cost_grading + cost_wrong_group_size + cost_wanted_not_in_group + cost_not_wanted_in_group


        if cost < min_cost:
            min_cost_groups = groups
            min_cost = cost
        
        
            min_cost_groups_str = ['-'.join(x) for x in min_cost_groups]

            result = {"assigned-groups": min_cost_groups_str,
                      "total-cost" : min_cost}

    

    
    
            yield (result)
        



In [ ]:
for result in solver_yield('test3.txt'):
    print("----- Latest solution:\n" + "\n".join(result["assigned-groups"]))
    print("\nAssignment cost: %d \n" % result["total-cost"])

----- Latest solution:
qian-li-wang5
fanjun-nthakurd-chen
djcran-sahmaini-wu
sulagaop-zheng-zhang3
zhao-zhou-zhang
vkvats-wang-li4
sun-feng

Assignment cost: 98 

----- Latest solution:
vkvats-zhao-zhou
nthakurd-wu-zhang3
djcran-sahmaini
li4-wang5-zhang
qian-li-feng
sulagaop-zheng-wang
fanjun-sun-chen

Assignment cost: 77 

----- Latest solution:
wang-chen
nthakurd-zheng-feng
fanjun-sun-li
zhou-li4-wang5
djcran-sahmaini-sulagaop
vkvats-qian-zhang
zhao-wu-zhang3

Assignment cost: 64 

----- Latest solution:
sahmaini-zhang
djcran-fanjun-li4
wu-zheng-zhang3
vkvats-zhou-feng
nthakurd-sun-wang
zhao-li-chen
sulagaop-qian-wang5

Assignment cost: 63 

----- Latest solution:
sahmaini-sulagaop-wang
sun-li-chen
vkvats-zhao-wang5
wu-feng
djcran-fanjun-zheng
nthakurd-zhou-zhang
qian-zhang3-li4

Assignment cost: 51 

----- Latest solution:
sulagaop-wu-wang
sahmaini-zhou
fanjun-li-zheng
nthakurd-li4-zhang
djcran-chen-wang5
zhao-feng-zhang3
vkvats-qian-sun

Assignment cost: 50 

----- Latest solution:

KeyboardInterrupt: 

# not as random search

Assign people what they want.  (random selection if 'zzz' or 'xxx'). then random groups of 3 for remaining. try 100 of these samples.

Then shuffle order of surveys and repeat above


In [10]:
def top_down_pref(people, want):
    picked_groups = []
    avail_people = list(people.copy())
    #     print(f'want {want}\n')



    for n in range(len(want)):
        try:
            grp = []
            for m in range (len(want[n])):
                if want[n][m] in avail_people:
    #                     print(f'{want[n][m]} available')
                    grp.append(want[n][m])
    #                     print(f'grp {grp}')
                    avail_people.remove(want[n][m])
    #                     print(f'avail_people {avail_people}')

                if want[n][m] in ['xxx','zzz']:
    #                     print(f'{want[n][m]} is wildcard')
                    rand_avail = np.random.choice(avail_people, 1)[0]
                    grp.append(rand_avail)
    #                     print(f'grp {grp}')
                    avail_people.remove(rand_avail)
    #                     print(f'avail_people {avail_people}')

            picked_groups.append(grp)
    #             print(f'picked_groups {picked_groups}\n')
        except:
            continue

    picked_groups = [x for x in picked_groups if x!= []]
    # print (f'picked_groups {picked_groups}')


    picked_people = [item for sublist in picked_groups for item in sublist]
    # print(f'picked_people {picked_people}')

    people_left = list(people.copy())
    [people_left.remove(picked_people[n]) for n in range(len(picked_people))]
    # print(f'people_left {people_left}')


    if people_left != []:
        picked_groups.append(people_left)
    #     print(f'picked groups {picked_groups}')



    n_group_check = [len(grp) for grp in picked_groups]
    # print(f'n_group_check {n_group_check}')
    if max(n_group_check)>3:
        group_to_split_idx = [n_group_check.index(x) for x in n_group_check if x>3][0]
    #     print(f'group_to_split_idx {group_to_split_idx}')
        group_to_split = picked_groups.pop(group_to_split_idx)
        n =3
        [picked_groups.append(group_to_split[i*n:(i+1)*n]) for i in range((len(group_to_split)+n-1)//n)]
#         print(f'picked_groups {picked_groups}')

    return picked_groups


In [123]:
def solver(filename):
    with open(filename) as f:
        lines = f.readlines()
    
    survey_text = [line.rstrip('\n') for line in lines]
    survey_data = np.array([entry.split() for entry in survey_text])


    min_cost = 100000000
    shuffles = 1
    for shuffles in range(1000):
  
        np.random.shuffle(survey_data)

        #survey inputs
        people = survey_data[:,0]
        want = [item.split('-') for item in survey_data[:,1]]
        not_want = [item.split(',') for item in survey_data[:,2]]
        want_group_size = [len(item) for item in want]

        # print(f'\npeople {people}')

        runs = 100
        for n in range(runs):
            groups = top_down_pref(people, want)
            # print(f'groups {groups}')

            # group stats
            n_groups = len(groups)
            
            groups_size = [len(group) for group in groups]

            people_group_mat = [[people[x] in group for group in groups] for x in range(len(people))]
            group_assigned = [people_group_mat[n].index(1) for n in range(len(people_group_mat))]
            people_assigned =[groups[n] for n in group_assigned]

            #calculate cost
            cost_grading = grading(groups)
            cost_wrong_group_size = wrong_group_size(group_assigned, want_group_size, groups_size)
            cost_wanted_not_in_group = wanted_not_in_assigned(people_assigned, want)
            cost_not_wanted_in_group = not_wanted_in_assigned(people_assigned, not_want)

            cost = cost_grading + cost_wrong_group_size + cost_wanted_not_in_group + cost_not_wanted_in_group

            if cost < min_cost:
                min_cost_groups = groups
                min_cost = cost
    #                 print(n, min_cost_groups, min_cost)

        min_cost_groups_str = ['-'.join(x) for x in min_cost_groups]
        # print (f'min_cost_group after {runs} random samples is {min_cost_groups_str, min_cost}')

        result = {"assigned-groups": min_cost_groups_str,
                        "total-cost" : min_cost}


    shuffles+=1



    return result


In [125]:
solver('test2.txt')

{'assigned-groups': ['wu-fanjun',
  'sun-qian-djcran',
  'zheng-li-zhao',
  'sulagaop-sahmaini-vkvats',
  'zhou-nthakurd'],
 'total-cost': 46.0}

## couldn't figure out how to yield with infinite loop in a way that passes the test, so it's set to run a for loop instead.  yields once every 100 iterations

In [126]:
def solver_yield(filename):
    with open(filename) as f:
        lines = f.readlines()
    
    survey_text = [line.rstrip('\n') for line in lines]
    survey_data = np.array([entry.split() for entry in survey_text])


    min_cost = 100000000
    
    shuffles = 1
    for shuffles in range(10):
  
        np.random.shuffle(survey_data)

        #survey inputs
        people = survey_data[:,0]
        want = [item.split('-') for item in survey_data[:,1]]
        not_want = [item.split(',') for item in survey_data[:,2]]
        want_group_size = [len(item) for item in want]

        # print(f'\npeople {people}')

        runs = 100
        for n in range(runs):
            groups = top_down_pref(people, want)
            # print(f'groups {groups}')

            # group stats
            n_groups = len(groups)

            groups_size = [len(group) for group in groups]

            people_group_mat = [[people[x] in group for group in groups] for x in range(len(people))]
            group_assigned = [people_group_mat[n].index(1) for n in range(len(people_group_mat))]
            people_assigned =[groups[n] for n in group_assigned]

            #calculate cost
            cost_grading = grading(groups)
            cost_wrong_group_size = wrong_group_size(group_assigned, want_group_size, groups_size)
            cost_wanted_not_in_group = wanted_not_in_assigned(people_assigned, want)
            cost_not_wanted_in_group = not_wanted_in_assigned(people_assigned, not_want)

            cost = cost_grading + cost_wrong_group_size + cost_wanted_not_in_group + cost_not_wanted_in_group

            if cost < min_cost:
                min_cost_groups = groups
                min_cost = cost
    #                 print(n, min_cost_groups, min_cost)

                min_cost_groups_str = ['-'.join(x) for x in min_cost_groups]
                # print (f'min_cost_group after {runs} random samples is {min_cost_groups_str, min_cost}')

                result = {"assigned-groups": min_cost_groups_str,
                        "total-cost" : min_cost}

        yield (result)
            
        shuffles += 1

        
                        


In [128]:
for result in solver_yield('test3.txt'):
    print("----- Latest solution:\n" + "\n".join(result["assigned-groups"]))
    print("\nAssignment cost: %d \n" % result["total-cost"])
    
    

----- Latest solution:
zhou-vkvats
zhang-sun-li
li4-zhang3-sulagaop
chen-djcran
zhao-qian-wang5
zheng-wang
fanjun-feng
wu-sahmaini
nthakurd

Assignment cost: 85 

----- Latest solution:
zhou-vkvats
zhang-sun-li
li4-zhang3-sulagaop
chen-djcran
zhao-qian-wang5
zheng-wang
fanjun-feng
wu-sahmaini
nthakurd

Assignment cost: 85 

----- Latest solution:
zhou-vkvats
zhang-sun-li
li4-zhang3-sulagaop
chen-djcran
zhao-qian-wang5
zheng-wang
fanjun-feng
wu-sahmaini
nthakurd

Assignment cost: 85 

----- Latest solution:
zhou-vkvats
zhang-sun-li
li4-zhang3-sulagaop
chen-djcran
zhao-qian-wang5
zheng-wang
fanjun-feng
wu-sahmaini
nthakurd

Assignment cost: 85 

----- Latest solution:
chen-zhao
wang5-fanjun-sulagaop
sahmaini
wang-zheng
li-sun-qian
nthakurd
zhang-zhou-feng
djcran-vkvats
li4-zhang3-wu

Assignment cost: 74 

----- Latest solution:
chen-zhao
wang5-fanjun-sulagaop
sahmaini
wang-zheng
li-sun-qian
nthakurd
zhang-zhou-feng
djcran-vkvats
li4-zhang3-wu

Assignment cost: 74 

----- Latest solution:

In [ ]:
# ok_to_assign
ok_people = [people[np.where(np.isin(people, not_want[n])==0)] for n in range (len(people))]
ok_people

[array(['djcran', 'sulagaop', 'fanjun', 'nthakurd', 'vkvats', 'zhao',
        'qian', 'sun', 'li', 'zhou', 'wu', 'zheng'], dtype='<U22'),
 array(['djcran', 'sahmaini', 'sulagaop', 'fanjun', 'nthakurd', 'vkvats',
        'zhao', 'qian', 'sun', 'li', 'zhou', 'wu', 'zheng'], dtype='<U22'),
 array(['djcran', 'sahmaini', 'sulagaop', 'fanjun', 'nthakurd', 'vkvats',
        'zhao', 'qian', 'sun', 'li', 'zhou', 'wu', 'zheng'], dtype='<U22'),
 array(['djcran', 'sahmaini', 'sulagaop', 'fanjun', 'vkvats', 'zhao',
        'qian', 'sun', 'li', 'zhou', 'wu', 'zheng'], dtype='<U22'),
 array(['sahmaini', 'sulagaop', 'nthakurd', 'vkvats', 'zhao', 'qian',
        'sun', 'li', 'zhou', 'wu', 'zheng'], dtype='<U22'),
 array(['djcran', 'sahmaini', 'sulagaop', 'fanjun', 'nthakurd', 'vkvats',
        'zhao', 'qian', 'sun', 'li', 'zhou', 'wu', 'zheng'], dtype='<U22'),
 array(['djcran', 'sahmaini', 'sulagaop', 'fanjun', 'nthakurd', 'vkvats',
        'zhao', 'qian', 'sun', 'li', 'zhou', 'wu', 'zheng'], dtype='<U

In [97]:
def solver(input_file):
    """
    1. This function should take the name of a .txt input file in the format indicated in the assignment.
    2. It should return a dictionary with the following keys:
        - "assigned-groups" : a list of groups assigned by the program, each consisting of usernames separated by hyphens
        - "total-cost" : total cost (time spent by instructors in minutes) in the group assignment
    3. Do not add any extra parameters to the solver() function, or it will break our grading and testing code.
    4. Please do not use any global variables, as it may cause the testing code to fail.
    5. To handle the fact that some problems may take longer than others, and you don't know ahead of time how
       much time it will take to find the best solution, you can compute a series of solutions and then
       call "yield" to return that preliminary solution. Your program can continue yielding multiple times;
       our test program will take the last answer you 'yielded' once time expired.
    """

    # Simple example. First we yield a quick solution
    yield({"assigned-groups": ["vibvats-djcran-zkachwal", "shah12", "vrmath"],
               "total-cost" : 12})

    # Then we think a while and return another solution:
    time.sleep(10)
    yield({"assigned-groups": ["vibvats-djcran-zkachwal", "shah12-vrmath"],
               "total-cost" : 10})

    # This solution will never befound, but that's ok; program will be killed eventually by the
    #  test script.
    while True:
        pass
    
    yield({"assigned-groups": ["vibvats-djcran", "zkachwal-shah12-vrmath"],
               "total-cost" : 9})

In [115]:
def solver (input_file):
    # Simple example. First we yield a quick solution
    
    cost = 1
    group = 'group'
    yield({"assigned-groups": ["vibvats-djcran-zkachwal", "shah12", "vrmath"],
           "total-cost" : cost})
    
    while cost > 0:
        cost +=1
    
    while True:
        time.sleep(5)
        yield({"assigned-groups": ["vibvats-djcran-zkachwal", "shah12", "vrmath"],
               "total-cost" : cost})

    
    

In [116]:
if __name__ == "__main__":
    if(len('XX') != 2):
        raise(Exception("Error: expected an input filename"))

    for result in solver('test1.txt'):
        print("----- Latest solution:\n" + "\n".join(result["assigned-groups"]))
        print("\nAssignment cost: %d \n" % result["total-cost"])
    

----- Latest solution:
vibvats-djcran-zkachwal
shah12
vrmath

Assignment cost: 1 

----- Latest solution:
vibvats-djcran-zkachwal
shah12
vrmath

Assignment cost: 2 

----- Latest solution:
vibvats-djcran-zkachwal
shah12
vrmath

Assignment cost: 2 

----- Latest solution:
vibvats-djcran-zkachwal
shah12
vrmath

Assignment cost: 2 

----- Latest solution:
vibvats-djcran-zkachwal
shah12
vrmath

Assignment cost: 2 



KeyboardInterrupt: 